In [39]:
%reset -f

In [40]:
import re

import asyncio
from functools import partial

In [41]:
loop = asyncio.get_event_loop()

In [42]:
%%html
<link rel="stylesheet" type="text/css" href="theme/solarized.css">

### ~~Winter~~ Lightning talk is coming...

<img src="img/awaiting2.png" width="45%", style="margin-left: 23%">

## Does this make you `awaitables?`

## PEP 492, 525

### Here's the plan:

In [43]:
list(map(
    partial(re.sub, r'^(__)', r'\1a'), 
    ['__iter__', '__next__', '__enter__', '__exit__']
))

['__aiter__', '__anext__', '__aenter__', '__aexit__']

### That's the way you do it

_(You play the guitar on the MTV)_

In [45]:
async def my_async_gen() -> str:
    """Async generator :)"""
    injected_value = None
    
    for value in '42,foo,bar'.split(','):
        
        injected_value = yield {'value': value, 'injected_value': injected_value}
        await asyncio.sleep(1)

In [46]:
async def async_loop(gen):
        # agen.__aiter__() returns an async iterator object
        async for return_value in gen:
            print(return_value['value'])
            

In [47]:
gen = my_async_gen()
res = loop.run_until_complete(async_loop(gen))

42
foo
bar


In [48]:
async def async_inject(gen):
    # First time need to send None
    for communicate_value in [None, 'injected-baz', 'injected-qux']:
        
        # agen.__anext__() returns an awaitable
        return_value = await gen.asend(communicate_value)
        
        print('{value}, {injected_value}'.format(**return_value))

# Rehydrate that generator
gen = my_async_gen()
res = loop.run_until_complete(async_inject(gen))

42, None
foo, injected-baz
bar, injected-qux


In [49]:
class AsyncContextManager:
    async def log(self, msg):
        print(msg)
        asyncio.sleep(0.5)
    
    async def __aenter__(self):
        await self.log('--- Entering context')

    async def __aexit__(self, exc_type, exc, tb):
        await self.log('--- Exiting context')

In [50]:
async def async_loop_wrapped(gen):
    # aenter / aexit are async!
    async with AsyncContextManager():
        # agen.__aiter__() returns an async iterator object

        async for return_value in gen:
            print(return_value['value'])

gen = my_async_gen()
res = loop.run_until_complete(async_loop_wrapped(gen))

--- Entering context
42
foo
bar
--- Exiting context


In [ ]:
# loop.close()

`pandriychenko@hbk.com`

`rs2@github`

<sub>presented on Windows</sub>☺

<img src="img/hands-up2.png">